In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
from rasterio.transform import from_origin
import geopandas as gpd
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
import glob


In [2]:
# Read the Parquet file
df = pd.read_parquet("wheat_pixel.parquet")


In [3]:
df

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14,timestep_15
0,30.240077,77.714064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.240077,77.714154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.240077,77.714243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.240077,77.714333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.240077,77.714423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50315695,29.580804,78.329410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315696,29.580804,78.329500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315697,29.580804,78.329589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315698,29.580804,78.329679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = df.replace(0, pd.NA)
data = df.dropna()
data

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14,timestep_15
2689,30.240077,77.955621,0.689183,0.763441,0.56076,0.329061,0.734121,0.618307,0.468442,0.572899,0.584461,0.412004,0.535559,0.555848,0.538966,0.548433,0.420332
2690,30.240077,77.955711,0.689183,0.745437,0.560284,0.329061,0.693106,0.618307,0.468442,0.582343,0.559974,0.412004,0.581933,0.568282,0.592775,0.580247,0.409467
2692,30.240077,77.955890,0.688323,0.723451,0.513051,0.295814,0.653905,0.637879,0.481199,0.613772,0.642257,0.487943,0.696744,0.678032,0.655233,0.758725,0.484156
2693,30.240077,77.955980,0.628198,0.700569,0.691801,0.245416,0.634085,0.584142,0.420202,0.438344,0.551674,0.401231,0.766369,0.555107,0.5682,0.63976,0.469838
9547,30.239987,77.955890,0.738962,0.791591,0.725837,0.280937,0.73527,0.702197,0.528455,0.703812,0.675507,0.476987,0.639552,0.722059,0.650919,0.696562,0.512339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.389372,0.445161,0.40315,0.341888,0.413263,0.243482,0.232194,0.454445,0.457061,0.384914,0.461203,0.147493,0.125387,0.100979,0.107644
50298002,29.580983,77.971611,0.409758,0.433245,0.326545,0.367507,0.426961,0.258675,0.245203,0.458836,0.446097,0.416694,0.508655,0.162067,0.147593,0.125903,0.107327
50298014,29.580983,77.972689,0.486664,0.518783,0.341455,0.400084,0.463271,0.290848,0.260388,0.444206,0.413858,0.410491,0.491796,0.460197,0.440876,0.35354,0.320082
50304784,29.580893,77.965053,0.604847,0.605565,0.391886,0.507123,0.522235,0.374431,0.196094,0.455135,0.464253,0.498973,0.558845,0.543086,0.505257,0.515521,0.521825


In [5]:
import pandas as pd

# List of columns to convert to numerical types
columns_to_convert = [
    'timestep_1', 'timestep_2', 'timestep_3', 'timestep_4', 'timestep_5', 
    'timestep_6', 'timestep_7', 'timestep_8', 'timestep_9', 'timestep_10', 
    'timestep_11', 'timestep_12', 'timestep_13', 'timestep_14', 'timestep_15'
]

# Convert the specified columns to numerical types
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Create a new DataFrame with the updated values
new_df = data.copy()

# Display the new DataFrame
print(new_df.head())


C:\Users\admin\AppData\Local\Temp\ipykernel_684\2796927958.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')


       latitude  longitude  timestep_1  timestep_2  timestep_3  timestep_4  \
2689  30.240077  77.955621    0.689183    0.763441    0.560760    0.329061   
2690  30.240077  77.955711    0.689183    0.745437    0.560284    0.329061   
2692  30.240077  77.955890    0.688323    0.723451    0.513051    0.295814   
2693  30.240077  77.955980    0.628198    0.700569    0.691801    0.245416   
9547  30.239987  77.955890    0.738962    0.791591    0.725837    0.280937   

      timestep_5  timestep_6  timestep_7  timestep_8  timestep_9  timestep_10  \
2689    0.734121    0.618307    0.468442    0.572899    0.584461     0.412004   
2690    0.693106    0.618307    0.468442    0.582343    0.559974     0.412004   
2692    0.653905    0.637879    0.481199    0.613772    0.642257     0.487943   
2693    0.634085    0.584142    0.420202    0.438344    0.551674     0.401231   
9547    0.735270    0.702197    0.528455    0.703812    0.675507     0.476987   

      timestep_11  timestep_12  timestep_13 

In [6]:
new_df

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14,timestep_15
2689,30.240077,77.955621,0.689183,0.763441,0.560760,0.329061,0.734121,0.618307,0.468442,0.572899,0.584461,0.412004,0.535559,0.555848,0.538966,0.548433,0.420332
2690,30.240077,77.955711,0.689183,0.745437,0.560284,0.329061,0.693106,0.618307,0.468442,0.582343,0.559974,0.412004,0.581933,0.568282,0.592775,0.580247,0.409467
2692,30.240077,77.955890,0.688323,0.723451,0.513051,0.295814,0.653905,0.637879,0.481199,0.613772,0.642257,0.487943,0.696744,0.678032,0.655233,0.758725,0.484156
2693,30.240077,77.955980,0.628198,0.700569,0.691801,0.245416,0.634085,0.584142,0.420202,0.438344,0.551674,0.401231,0.766369,0.555107,0.568200,0.639760,0.469838
9547,30.239987,77.955890,0.738962,0.791591,0.725837,0.280937,0.735270,0.702197,0.528455,0.703812,0.675507,0.476987,0.639552,0.722059,0.650919,0.696562,0.512339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.389372,0.445161,0.403150,0.341888,0.413263,0.243482,0.232194,0.454445,0.457061,0.384914,0.461203,0.147493,0.125387,0.100979,0.107644
50298002,29.580983,77.971611,0.409758,0.433245,0.326545,0.367507,0.426961,0.258675,0.245203,0.458836,0.446097,0.416694,0.508655,0.162067,0.147593,0.125903,0.107327
50298014,29.580983,77.972689,0.486664,0.518783,0.341455,0.400084,0.463271,0.290848,0.260388,0.444206,0.413858,0.410491,0.491796,0.460197,0.440876,0.353540,0.320082
50304784,29.580893,77.965053,0.604847,0.605565,0.391886,0.507123,0.522235,0.374431,0.196094,0.455135,0.464253,0.498973,0.558845,0.543086,0.505257,0.515521,0.521825


In [7]:
rename_dict = {f'timestep_{i}': f'Band_{i}' for i in range(1, 16)}

# Rename the columns
new_df.rename(columns=rename_dict, inplace=True)

In [8]:
Range = new_df[(new_df.Band_1 >= 0) & (new_df.Band_1 <= 1) 
            & (new_df.Band_2 >= 0) & (new_df.Band_2 <= 1) 
            & (new_df.Band_3 >= 0) & (new_df.Band_3 <= 1)
            & (new_df.Band_4 >= 0) & (new_df.Band_4 <= 1)
            & (new_df.Band_5 >= 0) & (new_df.Band_5 <= 1)
            & (new_df.Band_6 >= 0) & (new_df.Band_6 <= 1)
            & (new_df.Band_7 >= 0) & (new_df.Band_7 <= 1)
            & (new_df.Band_8 >= 0) & (new_df.Band_8 <= 1)
            & (new_df.Band_9 >= 0) & (new_df.Band_9 <= 1)
            & (new_df.Band_10 >= 0) & (new_df.Band_10 <= 1)
            & (new_df.Band_11 >= 0) & (new_df.Band_11 <= 1)
            & (new_df.Band_12 >= 0) & (new_df.Band_12 <= 1)
            & (new_df.Band_13 >= 0) & (new_df.Band_13 <= 1)
            & (new_df.Band_14 >= 0) & (new_df.Band_14 <= 1)
            & (new_df.Band_15 >= 0) & (new_df.Band_15 <= 1)
            
           ]

In [9]:
Range

,latitude,longitude,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12,Band_13,Band_14,Band_15
2689,30.240077,77.955621,0.689183,0.763441,0.560760,0.329061,0.734121,0.618307,0.468442,0.572899,0.584461,0.412004,0.535559,0.555848,0.538966,0.548433,0.420332
2690,30.240077,77.955711,0.689183,0.745437,0.560284,0.329061,0.693106,0.618307,0.468442,0.582343,0.559974,0.412004,0.581933,0.568282,0.592775,0.580247,0.409467
2692,30.240077,77.955890,0.688323,0.723451,0.513051,0.295814,0.653905,0.637879,0.481199,0.613772,0.642257,0.487943,0.696744,0.678032,0.655233,0.758725,0.484156
2693,30.240077,77.955980,0.628198,0.700569,0.691801,0.245416,0.634085,0.584142,0.420202,0.438344,0.551674,0.401231,0.766369,0.555107,0.568200,0.639760,0.469838
9547,30.239987,77.955890,0.738962,0.791591,0.725837,0.280937,0.735270,0.702197,0.528455,0.703812,0.675507,0.476987,0.639552,0.722059,0.650919,0.696562,0.512339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.389372,0.445161,0.403150,0.341888,0.413263,0.243482,0.232194,0.454445,0.457061,0.384914,0.461203,0.147493,0.125387,0.100979,0.107644
50298002,29.580983,77.971611,0.409758,0.433245,0.326545,0.367507,0.426961,0.258675,0.245203,0.458836,0.446097,0.416694,0.508655,0.162067,0.147593,0.125903,0.107327
50298014,29.580983,77.972689,0.486664,0.518783,0.341455,0.400084,0.463271,0.290848,0.260388,0.444206,0.413858,0.410491,0.491796,0.460197,0.440876,0.353540,0.320082
50304784,29.580893,77.965053,0.604847,0.605565,0.391886,0.507123,0.522235,0.374431,0.196094,0.455135,0.464253,0.498973,0.558845,0.543086,0.505257,0.515521,0.521825


In [10]:
wheat = Range.drop(['longitude','latitude'],axis=1)
wheat

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12,Band_13,Band_14,Band_15
2689,0.689183,0.763441,0.560760,0.329061,0.734121,0.618307,0.468442,0.572899,0.584461,0.412004,0.535559,0.555848,0.538966,0.548433,0.420332
2690,0.689183,0.745437,0.560284,0.329061,0.693106,0.618307,0.468442,0.582343,0.559974,0.412004,0.581933,0.568282,0.592775,0.580247,0.409467
2692,0.688323,0.723451,0.513051,0.295814,0.653905,0.637879,0.481199,0.613772,0.642257,0.487943,0.696744,0.678032,0.655233,0.758725,0.484156
2693,0.628198,0.700569,0.691801,0.245416,0.634085,0.584142,0.420202,0.438344,0.551674,0.401231,0.766369,0.555107,0.568200,0.639760,0.469838
9547,0.738962,0.791591,0.725837,0.280937,0.735270,0.702197,0.528455,0.703812,0.675507,0.476987,0.639552,0.722059,0.650919,0.696562,0.512339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.389372,0.445161,0.403150,0.341888,0.413263,0.243482,0.232194,0.454445,0.457061,0.384914,0.461203,0.147493,0.125387,0.100979,0.107644
50298002,0.409758,0.433245,0.326545,0.367507,0.426961,0.258675,0.245203,0.458836,0.446097,0.416694,0.508655,0.162067,0.147593,0.125903,0.107327
50298014,0.486664,0.518783,0.341455,0.400084,0.463271,0.290848,0.260388,0.444206,0.413858,0.410491,0.491796,0.460197,0.440876,0.353540,0.320082
50304784,0.604847,0.605565,0.391886,0.507123,0.522235,0.374431,0.196094,0.455135,0.464253,0.498973,0.558845,0.543086,0.505257,0.515521,0.521825


In [14]:
new_column_names = [4, 9, 19, 24, 39, 49, 99, 109, 119, 134, 144, 149, 159, 174, 179]

# Rename columns
wheat.columns = new_column_names

In [15]:
wheat

,4,9,19,24,39,49,99,109,119,134,144,149,159,174,179
2689,0.689183,0.763441,0.560760,0.329061,0.734121,0.618307,0.468442,0.572899,0.584461,0.412004,0.535559,0.555848,0.538966,0.548433,0.420332
2690,0.689183,0.745437,0.560284,0.329061,0.693106,0.618307,0.468442,0.582343,0.559974,0.412004,0.581933,0.568282,0.592775,0.580247,0.409467
2692,0.688323,0.723451,0.513051,0.295814,0.653905,0.637879,0.481199,0.613772,0.642257,0.487943,0.696744,0.678032,0.655233,0.758725,0.484156
2693,0.628198,0.700569,0.691801,0.245416,0.634085,0.584142,0.420202,0.438344,0.551674,0.401231,0.766369,0.555107,0.568200,0.639760,0.469838
9547,0.738962,0.791591,0.725837,0.280937,0.735270,0.702197,0.528455,0.703812,0.675507,0.476987,0.639552,0.722059,0.650919,0.696562,0.512339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.389372,0.445161,0.403150,0.341888,0.413263,0.243482,0.232194,0.454445,0.457061,0.384914,0.461203,0.147493,0.125387,0.100979,0.107644
50298002,0.409758,0.433245,0.326545,0.367507,0.426961,0.258675,0.245203,0.458836,0.446097,0.416694,0.508655,0.162067,0.147593,0.125903,0.107327
50298014,0.486664,0.518783,0.341455,0.400084,0.463271,0.290848,0.260388,0.444206,0.413858,0.410491,0.491796,0.460197,0.440876,0.353540,0.320082
50304784,0.604847,0.605565,0.391886,0.507123,0.522235,0.374431,0.196094,0.455135,0.464253,0.498973,0.558845,0.543086,0.505257,0.515521,0.521825


## heading dates 

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Double Gaussian function
def double_gaussian(x, a1, b1, c1, a2, b2, c2):
    return (a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) +
            a2 * np.exp(-(x - b2)**2 / (2 * c2**2)))

days = np.array(wheat.columns, dtype=float)  # Convert column headers to float
data = wheat.values.astype(float)  # Convert the data values to float

# Normalize the data
data_norm = (data - np.min(data, axis=1, keepdims=True)) / (np.max(data, axis=1, keepdims=True) - np.min(data, axis=1, keepdims=True))

# Fit the double Gaussian to each row
fitted_params = []
maxima_values = []
for i in range(data_norm.shape[0]):
    try:
        # Adjust initial guesses based on data characteristics
        a1_guess = max(data_norm[i])
        b1_guess = days[np.argmax(data_norm[i])]
        c1_guess = (days[-1] - days[0]) / 10  # Rough estimate for the width of the first peak
        a2_guess = max(data_norm[i]) / 2
        b2_guess = days[len(days) // 2]
        c2_guess = (days[-1] - days[0]) / 5  # Rough estimate for the width of the second peak

        bounds = (0, [np.inf, np.max(days), np.inf, np.inf, np.max(days), np.inf])
        popt, _ = curve_fit(double_gaussian, days, data_norm[i], p0=[a1_guess, b1_guess, c1_guess, a2_guess, b2_guess, c2_guess], bounds=bounds, maxfev=10000)
        fitted_params.append(popt)
        
        # Find maximum value of the fitted curve
        max_value = np.max(double_gaussian(days, *popt))
        maxima_values.append(max_value)
    except RuntimeError as e:
        # print(f"Fit did not converge for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        maxima_values.append(np.nan)
    except Exception as e:
        print(f"An error occurred for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        maxima_values.append(np.nan)

# Save maxima_values to an Excel file
maxima_days = []
for i in range(len(fitted_params)):
    if not np.isnan(fitted_params[i][0]):
        fitted_curve = double_gaussian(days, *fitted_params[i])
        max_index = np.argmax(fitted_curve)
        max_day = days[max_index]
        maxima_days.append([i + 1, max_day])
    else:
        maxima_days.append([i + 1, np.nan])

# Convert to DataFrame for easier saving
maxima_df = pd.DataFrame(maxima_days, columns=['Row', 'Max_Day'])

# Save to Parquet
maxima_df.to_parquet('Heading.parquet', index=False)
print("Maxima days saved to Heading.parquet file.")


Maxima days saved to Heading.parquet file.
